## HQC

In [821]:
class HQC:
    
    def __init__(self, n, k, delta, w, w_e, w_r, C_1, C_2):
        
        self.n = n
        self.k = k
        self.delta = delta
        self.w = w
        self.w_e = w_e
        self.w_r = w_r
        self.C_1 = C_1
        self.C_2 = C_2
        
        self.F = GF(2)
        self.R = PolynomialRing(self.F, 'x'); x = R.gen()
        self.S = QuotientRing(self.R, x**self.n - 1, 'a'); a = S.gen()
        
        self.pk, self.sk = self.KeyGen(self.n,self.k, self.delta, self.w, self.w_e, self.w_r)
        
        
    def KeyGen(self, n, k, delta, w, w_e, w_r):
        
        h = self.R.random_element(degree=(0,self.n))
        h = self.S(h)
        
        x = self.R.random_element(degree=(0,self.n))
        while (x.hamming_weight() != self.w):
            x = self.R.random_element(degree=(0,self.n))
        x = self.S(x)
        
        y = self.R.random_element(degree=(0,self.n))
        while (y.hamming_weight() != self.w):
            y = self.R.random_element(degree=(0,self.n))
        y = self.S(y)
        
        s = x + h * y
        
        pk = (h,s)
        sk = (x,y)
        
        return pk, sk
    
    def Encrypt(self, pk, m):
        
        e = self.R.random_element(degree=(0,self.n))
        while (e.hamming_weight() != self.w_e):
            e = self.R.random_element(degree=(0,self.n))
        e = self.S(e)
        
        r1 = self.R.random_element(degree=(0,self.n))
        while (r1.hamming_weight() != self.w_r):
            r1 = self.R.random_element(degree=(0,self.n))
        r1 = self.S(r1)
        
        r2 = self.R.random_element(degree=(0,self.n))
        while (r2.hamming_weight() != self.w_r):
            r2 = self.R.random_element(degree=(0,self.n))
        r2 = self.S(r2)
        
        errors = (self.sk[0] * r2 - r1 * self.sk[1] + e).list()
        errors_HW  = errors.count(1)
        print('x * r2 - r1 * y + e = ', errors_HW)
        print('delta = ', self.delta)
        
        u = r1 + pk[0] * r2
        
        if (self.C_1 == 'BCH' and self.C_2 == 'REP'):
            self.C = BCHREPCode(n_BCH = 15, m = 4, b = 1, D = 7, n_REP = 8)
            v = self.C.Encoding(m)
            v = self.S(v)
            v = v + pk[1] * r2 + e
        elif (self.C_1 == 'RS' and self.C_2 == 'RM'):
            self.C = RSRMCode(n_RS = 15, k_RS = 7, q_RS = 2**4, r_RM = 1, m_RM = 3)
            v = self.C.Encoding(m)
            
        else:
            raise ValueError('Wrong codes')
        
        return (u,v)
    
    def Decrypt(self, sk, c):
        
        d = c[1] - c[0] * sk[1]

        return self.C.Decoding(d.list())
        
        

In [822]:
code = HQC(n = 120, k = 5, delta = 56 , w = 20, w_e = 20, w_r = 20, C_1 = 'BCH', C_2 = 'REP')

In [823]:
c = code.Encrypt(code.pk, [1,1,1,0,1])
print(c)                       

x * r2 - r1 * y + e =  34
delta =  56
(a^89 + a^88 + a^87 + a^84 + a^82 + a^76 + a^73 + a^72 + a^71 + a^69 + a^68 + a^67 + a^63 + a^61 + a^60 + a^59 + a^54 + a^52 + a^49 + a^47 + a^38 + a^36 + a^35 + a^32 + a^30 + a^29 + a^28 + a^27 + a^26 + a^22 + a^18 + a^17 + a^15 + a^14 + a^13 + a^10 + a^7 + a^4 + a^3 + 1, a^119 + a^116 + a^114 + a^111 + a^108 + a^107 + a^106 + a^103 + a^102 + a^101 + a^100 + a^98 + a^96 + a^95 + a^94 + a^91 + a^89 + a^86 + a^85 + a^84 + a^83 + a^82 + a^81 + a^79 + a^78 + a^74 + a^71 + a^69 + a^68 + a^67 + a^63 + a^60 + a^59 + a^58 + a^56 + a^55 + a^53 + a^51 + a^50 + a^48 + a^47 + a^46 + a^42 + a^40 + a^31 + a^28 + a^24 + a^22 + a^21 + a^19 + a^18 + a^17 + a^15 + a^12 + a^8 + a^7 + a^6 + a^3)


In [824]:
d = code.Decrypt(code.sk, c)
print(d)

[1, 1, 1, 0, 1]


In [614]:
class BCHREPCode:
    
    def __init__(self, n_BCH, m, b, D, n_REP):
        
        self.q = 2
        self.n_BCH = n_BCH
        self.m = m
        self.b = b
        self.D = D
        self.n_REP = n_REP
            
        if (gcd(self.n_BCH, self.q) != 1):
            raise ValueError('Invalid input values: gcd(n,q) != 1.')
            
        if (self.n_BCH != self.q**self.m - 1):
            raise ValueError('Invalid input values: n != q^m - 1')
            
        
        # Initialize field
        self.BF = GF(self.q) # base field
        self.EF = GF(self.q**self.m) # extension field
        R.<x> = PolynomialRing(self.BF, 'x')
        self.R = R
        self.x = x
        self.R_RS = PolynomialRing(self.EF, 'X')
        self.alpha = self.EF.primitive_element()
        #self.root_sequence = [self.alpha**i for i in range(self.b,self.b+self.D-1)]
        
        # define RS parameters
        self.k_RS = self.n_BCH - self.D + 1
        self.tau_RS = floor( (self.D - 1) / 2)
        self.alpha_RS = vector([self.alpha**i for i in range(self.n_BCH)])
        self.G_RS = matrix(self.EF, self.k_RS, self.n_BCH, lambda i,j : self.alpha_RS[j]**i)
        
        # Constructing generator matrix
        self.cosets = self.cyclotomic_cosets(self.n_BCH, self.q, self.b, self.D)
        
        self.generator_poly = self.BCH_generator_polynomial(self.x, self.alpha, self.D, self.cosets)
        
        if not (self.generator_poly.divides(self.x**self.n_BCH - 1)):
            raise ValueError('generator_poly is not a generator polynomial')
            
        self.k = self.n_BCH - self.generator_poly.degree()
        
        self.G = matrix(self.BF, self.k, self.n_BCH, lambda i,j : self.generator_poly[(j+(self.n_BCH-i)) % self.n_BCH])
        
        
    def cyclotomic_cosets(self, n, q, b, D):
        # compute cyclotomic cosets
    
        cosets = []

        for i in range(b,b+D-1):
            coset = [(i * q**j) % n for j in range(0,n-1)]
            coset = list(set(coset))
            coset.sort()
            cosets.append(coset)
        return cosets

    def minimal_polynomial(self, coset, x, alpha):
        # compute minimal polynomial from one coset
        poly = 1
        for j in range(len(coset)):
            poly *= (x - alpha**coset[j])
        return poly

    def BCH_generator_polynomial(self, x, alpha, D, cosets):
        # compute generator polynomial
        poly = self.minimal_polynomial(cosets[0], x, alpha)
        for i in range(1,D-1):
            poly = LCM(poly,self.minimal_polynomial(cosets[i],x,alpha))
        
        return poly
    
    
    def Encoding(self, message, zeropad = True):
        
        rem = len(message) % self.k
        
        if rem != 0:
            if zeropad:
                message.extend([self.BF(0)]*(self.k-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(message), self.k):
            c.extend(self.EncodeChunkBCH(message[i:i+self.k]))
        
        return c
            
    def EncodeChunkBCH(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.BF, chunk) * self.G
        
        c_BCH_REP = self.EncodeChunkREP(c)
        
        return c_BCH_REP
    
    def EncodeChunkREP(self, chunk):
        
        c_BCH_REP = []
        
        for i in range(len(chunk)):
            for j in range(self.n_REP):
                c_BCH_REP.append(chunk[i])
                
        return c_BCH_REP
                
    
    def Decoding(self, r):
        
        # Check input size
        if len(r) % self.n_BCH != 0:
            raise ValueError('Invalid input size')
            
        c = []
        
        for i in range(0,len(r),self.n_REP):
            c.append(self.DecodeChunkREP(r[i:i+self.n_REP]))
        
        for i in range(len(c)):
            c[i] = self.EF(c[i])
            
        c_final = []
        
        for i in range(0,len(c),self.n_BCH):
            c_final.extend(self.BivariateInterpolation(c[i:i+self.n_BCH]))
            
        c_final = self.EncodingRS(c_final)
        
        c_final = self.DecodingBCH(c_final)
        
        return c_final
    
    
    def DecodingBCH(self, c):
        
        if len(c) % self.n_BCH != 0:
            raise ValueError('Invalid input size')
            
        for i in range(len(c)):
            c[i] = self.BF(c[i])
            
        c_final = []
        
        for i in range(0,len(c),self.n_BCH):
            c_final.extend(self.DecodeChunkBCH(c[i:i+self.n_BCH]))
            
        return c_final
    
    def DecodeChunkBCH(self, chunk):
        
        cols = self.G.pivots()
        G_independent = self.G.matrix_from_columns(cols)
        chunk_independent = [chunk[i] for i in cols]
        
        return vector(self.BF, chunk_independent) * G_independent.inverse()
    
    
    def EncodingRS(self, m, zeropad = True):
        
        rem = len(m) % self.k_RS
        
        if rem != 0:
            if zeropad:
                m.extend([self.EF(0)]*(self.k_RS-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(m), self.k_RS):
            c.extend(self.EncodeChunkRS(m[i:i+self.k_RS]))
        
        return c
            
    def EncodeChunkRS(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k_RS:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.EF, chunk) * self.G_RS
        return c
    
    def BivariateInterpolation(self, chunk):
        
        if len(chunk) != self.n_BCH:
            raise ValueError('Invalid chunk size')
            
        # Constructing matrices
        M1 = matrix(self.EF, self.n_BCH, self.tau_RS + self.k_RS, lambda i,j : self.alpha_RS[i]**j)
        M2 = matrix(self.EF, self.n_BCH, self.tau_RS + 1, lambda i,j : chunk[i] * self.alpha_RS[i]**j)
        M = M1.augment(M2)
        
        # Solving system
        RK = M.right_kernel()
        
        if len(RK.basis()) == 0:
            return(None)
        
        sol = RK.basis()[0]

        # Constructing Q0 and Q1 polynomials
        Q0 = self.R_RS(list(sol[:self.tau_RS+self.k_RS]))
        Q1 = self.R_RS(list(sol[self.tau_RS+self.k_RS:]))

        # Calculating -Q0/Q1
        q, r = Q0.quo_rem(Q1)

        if r != 0:
            #print('Non-zero remainder (possibly >tau errors). Returning None')
            return(None)

        out = []

        out.extend((-q).list())
        out.extend([self.EF(0)]*(self.k_RS-len(out)))

        return out
    
    def DecodeChunkREP(self, chunk):
        if chunk.count(1) >= ceil(self.n_REP / 2):
            return 1
        else:
            return 0